# Anwendung: Energiespeicherproblem mit rollierendem Zeitfenster
Wir betrachten das Batteriespeicherproblem {ref}`subsec:energiespeicher`.

<div>
<img src="./bilder/BeispielBatterie1.png" width="750"/>
</div>

Betrachten wir zunächst noch einmal die Situation aus dem Beispiel. Gegeben seien:

- Ein betrachteter Zeithorizont $\{1,2, \ldots, 24\}$ von einem Tag.
- Der Verbrauch $d_t$ zum Zeitpunkt $t$. 
- Der Strompreis $p_t$ zum Zeitpunkt $t$.

Weiterhin ist die Kapazität $s_{max}=20 kWh$ sowie der initiale Ladestand $s_0=0.0 kWh$ des installierten Batteriespeichers gegeben. Betrachten wir zunächst den Preisverlauf über einen Tag:

In [103]:
import plotly.express as px
import pandas as pd

df = pd.read_csv("Daten/strom_actuals.csv")
df.index = range(1,25)
px.line(data_frame=df, x="Datum (MEZ)", y="Preis [EUR/kWh]")

Wir sehen, dass der Preis zwischen -1 Cent am Vormittag und +14 Cent gegen Abend schwankt. Negative Preise können auftreten, wenn sehr viel Energie vorhanden ist (viel Wind und Sonne). In diesem Fall erhält man Geld, wenn man seine Stromproduktion einstellt oder Strom verbraucht, damit das Netz stabil bleibt.

Wir formulieren zunächst das Problem mit den tatsächlichen Preisen. Damit erhalten wir eine untere Schranke für die erzielbaren Stromkosten an diesem Tag. Untere Schranke, da wir in diesem Fall vollständige Information über den Preis haben und so unser Verhalten optimal daran anpassen können. In der Praxis kann dies natürlich nur im Nachhinein geschehen (wie viel hätte ich sparen können).

In [107]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np

H = df.index
s_0 = 0.0
s_max = 20

m = gp.Model("Energiespeicher (deterministisch)")

# Variablen
k = m.addVars(H, vtype=GRB.CONTINUOUS, lb=0.0)
s_in = m.addVars(H, vtype=GRB.CONTINUOUS, lb=0.0)
s_out = m.addVars(H, vtype=GRB.CONTINUOUS, lb=0.0)
s = m.addVars(np.hstack([0,H]), vtype=GRB.CONTINUOUS, lb=0.0, ub=s_max)

# Zielfunktion
m.setObjective(gp.quicksum(df.loc[t,"Preis [EUR/kWh]"]*k[t] for t in H), GRB.MINIMIZE)

# Constraints

# Bilanzgleichungen Bedarf
m.addConstrs((k[t] - s_in[t] + s_out[t] == df.loc[t,"Last [kWh]"] for t in H))

# Bilanzgleichungen Batteriestand
m.addConstrs((s[t-1] + s_in[t] - s_out[t] == s[t] for t in H))

# Startwert für Batteriestand
m.addConstr(s[0] == s_0)

m.optimize()


Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (linux64 - "Linux Mint 21.3")

CPU model: 11th Gen Intel(R) Core(TM) i5-1145G7 @ 2.60GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 49 rows, 97 columns and 169 nonzeros
Model fingerprint: 0x4aad8b28
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-04, 1e-01]
  Bounds range     [2e+01, 2e+01]
  RHS range        [8e+01, 1e+02]
Presolve removed 49 rows and 97 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    7.4893916e+01   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective  7.489391634e+01


Der optimale Stromkostenpreis für den Tag liegt bei 74.81 EUR. Wir speichern die optimalen Werte der Variablen im DataFrame ab und plotten die Lösung. Wir sehen, dass in drei Zeitperioden die Batterie voll geladen wird (niedriger Strompreis) und während eines höheren Strompreises wieder entladen wird. Der Wert von 74.81 EUR ist dabei der bestmögliche Wert, den man erzielen kann. Dabei muss man den Strompreis allerdings im Voraus genau kennen.

In [114]:
df["Netzbezug [kWh]"] = [k[t].x for t in H]
df["Ladung [kWh]"] = [s_in[t].x for t in H]
df["Entladung [kWh]"] = [s_out[t].x for t in H]
df["Batteriestand [kWh]"] = [s[t].x for t in H]

px.line(data_frame=df, x="Datum (MEZ)", y=["Netzbezug [kWh]","Ladung [kWh]", "Entladung [kWh]"])

Wir betrachten nun den realistischen Fall, dass wir lediglich eine Vorhersage für Strompreis haben und basierend darauf entscheiden müssen, ob es sich lohnt, jetzt die Batterie zu laden oder zu entladen. Wir gehen davon aus, dass wir zu jedem Zeitpunkt $\tau$ eine aktualisierte Vorhersage bekommen. 

In [148]:
df_forecast = pd.read_csv("Daten/strom_forecast.csv")
df_forecast["tau"] = pd.to_datetime(df_forecast["tau"])
df_forecast

,Datum (MEZ),Preis [EUR/kWh],Last [kWh],Preis Forecast [EUR/kWh],tau
0,2020-05-01 00:00:00+01:00,0.02675,87.8690,0.035598,2020-05-01 00:00:00+01:00
1,2020-05-01 01:00:00+01:00,0.01910,84.6616,0.025157,2020-05-01 00:00:00+01:00
2,2020-05-01 02:00:00+01:00,0.01315,83.2734,-0.034439,2020-05-01 00:00:00+01:00
3,2020-05-01 03:00:00+01:00,0.00780,82.9602,-0.028081,2020-05-01 00:00:00+01:00
4,2020-05-01 04:00:00+01:00,0.01230,82.4234,0.018148,2020-05-01 00:00:00+01:00
...,...,...,...,...,...
595,2020-05-02 19:00:00+01:00,0.14020,105.3522,0.277027,2020-05-02 00:00:00+01:00
596,2020-05-02 20:00:00+01:00,0.13185,101.8214,0.132304,2020-05-02 00:00:00+01:00
597,2020-05-02 21:00:00+01:00,0.12190,99.9364,0.450607,2020-05-02 00:00:00+01:00
598,2020-05-02 22:00:00+01:00,0.10245,94.4658,0.171091,2020-05-02 00:00:00+01:00


Wir schauen uns eine Vorhersage und den zugehörigen tatsächlichen Preis für ein gegebenes $\tau$ an. Anfangs stimmt die Vorhersage recht gut mit den aktuellen Werten überein. Je weiter in die Zukunft die Vorhersage reicht, desto größer sind die Abweichungen.

In [149]:
one_forecast = df_forecast[df_forecast["tau"]=="2020-05-01 00:00:00+01:00"]
px.line(data_frame=one_forecast, x="Datum (MEZ)", y=["Preis [EUR/kWh]", "Preis Forecast [EUR/kWh]"])

Wenn wir nun die Vorhersage zu einem anderen Zeitpunkt betrachten, z.B. 3 Stunden später, so ist die Vorhersage für z.B. 4 und 5 Uhr genauer, da diese nun näher am Zeitpunkt der Vorhersage liegen. Dies möchten wir bei der Optimierung ausnutzen.

In [150]:
one_forecast = df_forecast[df_forecast["tau"]=="2020-05-01 03:00:00+01:00"]
px.line(data_frame=one_forecast, x="Datum (MEZ)", y=["Preis [EUR/kWh]", "Preis Forecast [EUR/kWh]"])

Im folgenden implementieren wir die Schleife wie in {prf:ref}`alg:mpc` beschrieben mit einer Änderung: Damit die Lösung mit der deterministischen Lösung vergleichbar ist, wird der Vorhersagezeitraum mit jeder Stunde verkürzt, so dass man am Ende nur 1 Stunde in die Zukunft schaut. Damit wird verhindert, dass am Ende noch Strom für späteren Verbrauch gespeichert ist (der Zeithorizont ist ja zu Ende). Bei fortlaufenden Prozessen würde man in der Praxis aber ein konstantes Zeitfenster wählen, da es oft kein vorgegebenes Ende des Optimierungszeitraumes gibt. 

In [152]:
# Unterdrücke Gruobi Output
env = gp.Env(empty=True)
env.setParam("OutputFlag",0)
env.start()


tau0 = df_forecast["tau"].min()
s_0 = 0.0
s_max = 20

# Hier speichern wir von jedem gelösten MILP den ersten Zeitschritt
solution = []

# Iteriere über Stunden eines Tages. In jeder Stunde wird ein neues Problem gelöst.
for tau in range(24):
    # Zeithorizont für neues Problem
    H = range(tau+1, 25)

    # Schneide Problemdaten
    df_current = df_forecast[df_forecast["tau"] == tau0 + pd.Timedelta(hours=tau)]
    df_current.index = range(tau+1, tau+25)

    # Neues Problem
    m_roll = gp.Model("Energiespeicher (rollierend)", env=env)

    # Variablen
    k = m_roll.addVars(H, vtype=GRB.CONTINUOUS, lb=0.0)
    s_in = m_roll.addVars(H, vtype=GRB.CONTINUOUS, lb=0.0)
    s_out = m_roll.addVars(H, vtype=GRB.CONTINUOUS, lb=0.0)
    s = m_roll.addVars(np.hstack([tau,H]), vtype=GRB.CONTINUOUS, lb=0.0, ub=s_max)

    # Zielfunktion
    m_roll.setObjective(gp.quicksum(df_current.loc[t,"Preis Forecast [EUR/kWh]"]*k[t] for t in H), GRB.MINIMIZE)

    # Constraints

    # Bilanzgleichungen Bedarf
    m_roll.addConstrs((k[t] - s_in[t] + s_out[t] == df_current.loc[t,"Last [kWh]"] for t in H))

    # Bilanzgleichungen Batteriestand
    m_roll.addConstrs((s[t-1] + s_in[t] - s_out[t] == s[t] for t in H))

    # Startwert für Batteriestand
    m_roll.addConstr(s[tau] == s_0)

    # Starte Gurobi
    m_roll.optimize()

    # Behalte nur den ersten Zeitschritt der Lösung
    solution.append({"Datum (MEZ)": tau0 + pd.Timedelta(hours=tau), 
                     "Netzbezug [kWh]": k[tau+1].x,
                     "Ladung [kWh]": s_in[tau+1].x,
                     "Entladung [kWh]": s_out[tau+1].x,
                     "Batteriestand [kWh]": s[tau+1].x,
                     "Preis [EUR/kWh]": df.loc[tau+1,"Preis [EUR/kWh]"],
                     "Last [kWh]": df.loc[tau+1,"Last [kWh]"]})
    
    # Neuer Startwert für den Batterieladestand
    s_0 = s[tau+1].x

# Setze Lösung zusammen
df_rolling = pd.DataFrame.from_records(solution)


Wir schauen uns nun die durch diese Strategie tatsächlich erzielbare Lösung an und vergleichen sie mit der theoretisch besten erzielbaren Lösung. Wir betrachten:
- Die tatsächlich erzielten Kosten
- Die Entscheidungen zum Laden und Entladen der Batterie

In [160]:
print(f"Tatsächlich erzielte Stromkosten: {round(sum(df_rolling['Netzbezug [kWh]']*df_rolling['Preis [EUR/kWh]']),2)} EUR.")
print(f"Beste erzielbare Stromkosten: {round(sum(df['Netzbezug [kWh]']*df['Preis [EUR/kWh]']),2)} EUR.")

Tatsächlich erzielte Stromkosten: 75.65 EUR.
Beste erzielbare Stromkosten: 74.89 EUR.


Wir erklärt sich dieser Unterschied? Es kommt durch die verschiedenen Entscheidungen, wann der Speicher geladen bzw. entladen wurde.

In [163]:
px.line(data_frame=df_rolling, x="Datum (MEZ)", y=["Netzbezug [kWh]","Ladung [kWh]", "Entladung [kWh]"], title="Entscheidung basierend auf rollierenden Vorhersagen")

In [164]:
px.line(data_frame=df, x="Datum (MEZ)", y=["Netzbezug [kWh]","Ladung [kWh]", "Entladung [kWh]"], title="Bestmögliche Entscheidung")